In [7]:
# testing out ipyleaflet on here, because shiny is problems
# see example https://github.com/jupyter-widgets/ipyleaflet/blob/master/examples/GeoData_on_hover.ipynb

from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas as gpd
import pandas as pd
import json

from pathlib import Path

In [2]:
gdf = gpd.read_file(Path.cwd().joinpath('data').joinpath('clean_seattle_zipcodes.geojson'))


In [114]:
gdf.columns

Index(['id', 'zipcode', 'geometry'], dtype='object')

In [117]:
gdf.zipcode = gdf.zipcode.astype('int64')

In [126]:
gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [8]:
df = pd.read_csv(Path.cwd().joinpath('data').joinpath('seattle_dogs_single_breed.csv'))


In [78]:
df.columns

Index(['zip', 'breed', 'count'], dtype='object')

In [9]:
df.head()

,zip,breed,count
0,98101,airedale terrier,1
1,98101,akita,1
2,98101,american eskimo,1
3,98101,australian shepherd,4
4,98101,basenji,1


In [ ]:
# so now, I would want zip, and in the RHS the dict of breeds & counts
def data_prep_dogs_to_dict(df):
   """
   Create a dictionary of breed counts per zip code.
   - top breeds in each zip code
   - able to extract number of a specific breed in each zip code

   Returns dict of a dict: {zipcode:{breed:count})
   """

   grouped = df.groupby('zip')
   breed_counts_by_zip = {} # dict of a dict:  {zipcode:{breed:count})
   for i, (name, df) in enumerate(grouped):
      temp_dict = {} # note, there may be some null /empty dicts!
      breed_count = df[['breed', 'count']].to_dict(orient='records') # list of breeds

      # Let's sort the dictionaries by count, instead of alpha

      for elem_dict in breed_count:
         # e.g. unpack {'breed': 'australian shepherd', 'count': 4} to {'australian shepherd':4}
         temp_dict.update({elem_dict.get('breed'): elem_dict.get('count')})
         
      # Let's sort the dictionaries by count, instead of alphabetically
      breed_counts_by_zip[name] = dict(sorted(temp_dict.items(), key=lambda item: item[1], reverse=True))
      
   return breed_counts_by_zip

zipdict = data_prep_dogs_to_dict(df)

In [ ]:
# type(pd.DataFrame(index=zipdict.keys(), data=zipdict.values()).loc[98101])
# .loc will output a series.

#(wrap in str())
# following will return first 5 elements by count for each zip:
# pd.DataFrame(index=zipdict.keys(), data=zipdict.values()).loc[98101].rename(index='count').sort_values(ascending=False)[:5].reset_index().to_string()


'                  index  count\n0        french bulldog    7.0\n1              shih tzu    7.0\n2      golden retriever    6.0\n3  short coat chihuahua    6.0\n4    labrador retriever    5.0'

In [ ]:
# following will return first 5 elements by count for a specific zip, as a dataframe or dict, (or string is wraapped with str)
str(pd.DataFrame(index=zipdict.keys(), data=zipdict.values()).loc[98101].rename(index='count').sort_values(ascending=False)[:5].to_dict())

"{'french bulldog': 7.0, 'shih tzu': 7.0, 'golden retriever': 6.0, 'short coat chihuahua': 6.0, 'labrador retriever': 5.0}"

In [ ]:
# top_breeds_by_zip_df.to_csv('top_5_breeds_by_zip.csv', index=False)

In [89]:
str(pd.DataFrame(index=zipdict.keys(), data=zipdict.values()).loc[98101].rename(index='count').sort_values(ascending=False)[:5].to_dict( ))

"{'french bulldog': 7.0, 'shih tzu': 7.0, 'golden retriever': 6.0, 'short coat chihuahua': 6.0, 'labrador retriever': 5.0}"

In [ ]:



# okay, this is kind of ugly, but here we go:
top_breeds_by_zip_df = pd.DataFrame()

# df is the dog containing data
for zipcode in df.zip.unique():
    zip_top_dogs_str = str(pd.DataFrame(index=zipdict.keys(), data=zipdict.values()).loc[zipcode].rename(index='count').sort_values(ascending=False)[:5].to_dict())
    top_breeds_by_zip_df = pd.concat([top_breeds_by_zip_df, pd.DataFrame(data={'zip': [zipcode], 'top_5_breeds': [zip_top_dogs_str]})], axis=0)

In [ ]:
top_breeds_by_zip_df.head() # looks like that works 

,zip,top_5_breeds
0,98101,"{'french bulldog': 7.0, 'shih tzu': 7.0, 'gold..."
0,98102,"{'labrador retriever': 29.0, 'golden retriever..."
0,98103,"{'labrador retriever': 74.0, 'golden retriever..."
0,98104,"{'short coat chihuahua': 10.0, 'miniature long..."
0,98105,"{'labrador retriever': 54.0, 'golden retriever..."


In [116]:
top_breeds_by_zip_df.dtypes

zip              int64
top_5_breeds    object
dtype: object

In [119]:
# join with gdf, so we can show top breeds

gdf = gdf.merge(top_breeds_by_zip_df, how='inner', left_on='zipcode', right_on='zip')

In [120]:
gdf.head()

,id,zipcode,geometry,zip,top_5_breeds
0,98391,98391,"MULTIPOLYGON (((-122.14398 47.25753, -122.1439...",98391,"{'siberian husky': 1.0, 'french bulldog': nan,..."
1,98136,98136,"MULTIPOLYGON (((-122.40573 47.55960, -122.4039...",98136,"{'labrador retriever': 42.0, 'golden retriever..."
2,98144,98144,"MULTIPOLYGON (((-122.32061 47.59178, -122.3205...",98144,"{'labrador retriever': 41.0, 'golden retriever..."
3,98146,98146,"MULTIPOLYGON (((-122.39602 47.51371, -122.3956...",98146,"{'labrador retriever': 14.0, 'golden retriever..."
4,98155,98155,"MULTIPOLYGON (((-122.33056 47.75442, -122.3305...",98155,"{'boxer': 1.0, 'miniature smooth haired dachsh..."


In [121]:
# let's get rid of the extra columns
gdf = gdf[['zip', 'geometry', 'top_5_breeds']]

In [ ]:
# Okay, mapping stuff here.

In [122]:
m = Map(center=(47.6062, -122.33207), zoom=10)
geo_data = GeoData(
    geo_dataframe=gdf,
    style={
        "color": "black",
        "fillColor": "#366370",
        "opacity": 0.05,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0.3,
    },
    hover_style={"fillColor": "#b08a3e", "fillOpacity": 0.1},
)
m.add(geo_data)
m

Map(center=[47.6062, -122.33207], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [123]:
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl, GeoJSON


html = HTML("""Hover over a zip""")
html.layout.margin = "0px 20px 20px 20px"
control = WidgetControl(widget=html, position="topright")
m.add(control)

Map(bottom=91752.0, center=[47.6062, -122.33207], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [ ]:
def update_html(feature, **kwargs):
    html.value = f"""{feature["properties"]}
    """
    
geo_data.on_hover(update_html)

# yay this works